## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-06-16-52-36 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/06/us/nat...
1,2025-10-06-16-48-36 +0000,bbc,Sudan militia leader convicted of war crimes d...,https://www.bbc.com/news/articles/cly1egd5d0vo...
2,2025-10-06-16-43-00 +0000,wsj,Illinois Sues to Block Trump’s National Guard ...,https://www.wsj.com/us-news/illinois-trump-chi...
3,2025-10-06-16-39-22 +0000,nypost,Tesla teases fans with Tuesday event — here’s ...,https://nypost.com/2025/10/06/business/elon-mu...
4,2025-10-06-16-39-00 +0000,wsj,Trio Wins Nobel Prize in Medicine for Discover...,https://www.wsj.com/science/trio-wins-nobel-pr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,38
196,new,17
234,gaza,15
229,shutdown,13
228,government,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
241,2025-10-05-18-37-30 +0000,latimes,Judge blocks Trump from deploying California N...,https://www.latimes.com/world-nation/story/202...,87
86,2025-10-06-13-03-45 +0000,nypost,Inside Trump’s blunt exchange with Netanyahu a...,https://nypost.com/2025/10/06/world-news/insid...,84
164,2025-10-06-07-17-48 +0000,nypost,Federal judge blocks Trump’s National Guard de...,https://nypost.com/2025/10/06/us-news/federal-...,82
198,2025-10-05-23-53-33 +0000,bbc,Trump urges mediators to 'move fast' as key Ga...,https://www.bbc.com/news/articles/cj3y6g43248o...,80
179,2025-10-06-03-00-00 +0000,wsj,High Stakes for Trump in Supreme Court’s New Term,https://www.wsj.com/us-news/law/supreme-court-...,79


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
241,87,2025-10-05-18-37-30 +0000,latimes,Judge blocks Trump from deploying California N...,https://www.latimes.com/world-nation/story/202...
93,62,2025-10-06-12-42-08 +0000,nypost,Bari Weiss to run new debate show at CBS News ...,https://nypost.com/2025/10/06/media/bari-weiss...
86,46,2025-10-06-13-03-45 +0000,nypost,Inside Trump’s blunt exchange with Netanyahu a...,https://nypost.com/2025/10/06/world-news/insid...
69,44,2025-10-06-14-04-48 +0000,nypost,Supreme Court rejects Ghislaine Maxwell’s appe...,https://nypost.com/2025/10/06/us-news/supreme-...
30,41,2025-10-06-15-51-00 +0000,wsj,Funding for federal agencies lapsed beginning ...,https://www.wsj.com/politics/policy/federal-go...
203,30,2025-10-05-23-07-07 +0000,bbc,Gisèle Pelicot to face one of her rapists in c...,https://www.bbc.com/news/articles/cdx2d9lynk1o...
1,29,2025-10-06-16-48-36 +0000,bbc,Sudan militia leader convicted of war crimes d...,https://www.bbc.com/news/articles/cly1egd5d0vo...
75,29,2025-10-06-13-55-45 +0000,nypost,"Ukraine strikes Russian ammo plant, oil termin...",https://nypost.com/2025/10/06/world-news/ukrai...
214,29,2025-10-05-20-59-08 +0000,nypost,Jersey City Mayor Steve Fulop being eyed for b...,https://nypost.com/2025/10/05/us-news/jersey-c...
108,28,2025-10-06-11-05-26 +0000,nypost,"Three scientists — 2 Americans, 1 Japanese — t...",https://nypost.com/2025/10/06/world-news/three...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
